# Managing ML workflows with AWS Step Functions and the Data Science SDK

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
!mkdir -p tmp

In [ ]:
g = "raw.githubusercontent.com"
p = "PacktPublishing"
a = "Amazon-SageMaker-Cookbook"
mc = "master/Chapter01"

path = f"https://{g}/{p}/{a}/{mc}/files"

In [ ]:
fname = "management_experience_and_salary.csv"

!wget -P tmp {path}/{fname}

In [ ]:
import pandas as pd
filename = f"tmp/{fname}"
df_all_data = pd.read_csv(filename)

In [ ]:
df_all_data

In [ ]:
from sklearn.model_selection import train_test_split

dad = df_all_data

X = dad['management_experience_months'].values 
y = dad['monthly_salary'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, random_state=0
)

In [ ]:
import pandas as pd

df_training_data = pd.DataFrame({ 
    'monthly_salary': y_train, 
    'management_experience_months': X_train
})

df_training_data

In [ ]:
df_training_data.to_csv(
    'tmp/training_data.csv', 
    header=False, index=False
)

In [ ]:
s3_bucket = 'sagemaker-cookbook-bucket'
prefix = 'chapter09'

In [ ]:
tn = "training_data.csv"
source = f"tmp/{tn}"
dest = f"s3://{s3_bucket}/{prefix}/input/{tn}"

!aws s3 cp {source} {dest}

In [ ]:
import sagemaker 
import boto3
from sagemaker import get_execution_role 

role = get_execution_role()
session = sagemaker.Session()
region_name = boto3.Session().region_name

In [ ]:
training_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/training_data.csv" 
training_s3_output_location = f"s3://{s3_bucket}/{prefix}/output/"

In [ ]:
from sagemaker.inputs import TrainingInput

train = TrainingInput(
    training_s3_input_location, 
    content_type="text/csv"
)

In [ ]:
from sagemaker.image_uris import retrieve 

container = retrieve(
    "linear-learner", 
    region_name, "1"
)

container

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path=training_s3_output_location,
    sagemaker_session=session
)

In [ ]:
estimator.set_hyperparameters(
    predictor_type='regressor', 
    mini_batch_size=4
)

In [ ]:
!pip -q install --upgrade stepfunctions

In [ ]:
execution_role = 'arn:aws:iam::581320662326:role/test-002'

In [ ]:
from stepfunctions.inputs import ExecutionInput

In [ ]:
execution_input = ExecutionInput(
    schema={ 
        'ModelName': str,
        'EndpointName': str,
        'JobName': str
    }
)

ei = execution_input

In [ ]:
from stepfunctions.steps import TrainingStep

In [ ]:
training_step = TrainingStep(
    'Training Step', 
    estimator=estimator,
    data={
        'train': train
    },
    job_name=ei['JobName']
)

In [ ]:
from stepfunctions.steps import ModelStep

In [ ]:
model_step = ModelStep(
    'Model Step',
    model=training_step.get_expected_model(),
    model_name=ei['ModelName']  
)

In [ ]:
from stepfunctions.steps import EndpointConfigStep

In [ ]:
endpoint_config_step = EndpointConfigStep(
    "Create Endpoint Configuration",
    endpoint_config_name=ei['ModelName'],
    model_name=ei['ModelName'],
    initial_instance_count=1,
    instance_type='ml.m5.xlarge'
)

In [ ]:
from stepfunctions.steps import EndpointStep

In [ ]:
endpoint_step = EndpointStep(
    "Deploy Endpoint",
    endpoint_name=ei['EndpointName'],
    endpoint_config_name=ei['ModelName']
)

In [ ]:
from stepfunctions.steps import Chain

In [ ]:
workflow_definition = Chain([
    training_step,
    model_step,
    endpoint_config_step,
    endpoint_step
])

In [ ]:
import uuid

uuid.uuid4().hex

In [ ]:
def generate_random_string():
    return uuid.uuid4().hex

grs = generate_random_string

In [ ]:
import uuid
from stepfunctions.workflow import Workflow

workflow = Workflow(
    name='{}-{}'.format('Workflow', grs()),
    definition=workflow_definition,
    role=execution_role,
    execution_input=execution_input
)

In [ ]:
workflow.create()

In [ ]:
execution = workflow.execute(
    inputs={
        'JobName': 'll-{}'.format(grs()),
        'ModelName': 'll-{}'.format(grs()),
        'EndpointName': 'll-{}'.format(grs())
    }
)

In [ ]:
execution.list_events()

In [ ]:
import pandas as pd

In [ ]:
events = execution.list_events()
pd.json_normalize(events)

In [ ]:
workflow.__dict__

In [ ]:
print(workflow.definition.to_json(pretty=True))